# LDPC Code Generation and Message Passing Decoding for Binary Symmetric, Binary Erasure, and BIAWGN Channels

In [2]:

using Distributions
using LDPC
using PyPlot

### In this segment, we approximate the erasure threshold for a given LDPC code using binary search.

In [3]:
#First we choose the variable and check node degrees for a regular (3,6) code
dv=3
dc=6

#Now we represent the lambda and rho polynomials as arrays
lam=zeros(Float64,1,2)
lam[:,1]=[1]
lam[:,2]=[2]

rho=zeros(Float64,1,2)
rho[:,1]=[1]
rho[:,2]=[5]

#note this is an approximation, and to increase the # of significant digits, gradually increase the last two entries
println("The erasure threshold for a ($dv,$dc) ldpc code is approximately $(calc_erasure_thresh(lam,rho,30,1000000))")


The erasure threshold for a (3,6) ldpc code is approximately 0.4294398142956197


In [4]:
#Now we estimate the erasure threshold for an irregular ldpc code from 
#Exact thresholds for low-density parity-check codes over the binary erasure channel
#Jianjun Mu, Xiaopeng Jiao, Xinmei Wang 
#http://www.sciencedirect.com/science/article/pii/S1002007109001051
#(lambda",rho") in table 1
ilam=zeros(Float64,3,2)
ilam[:,1]=[0.4706;0.2353;0.2941]
ilam[:,2]=[2;7;29]

irho=zeros(Float64,2,2)
irho[:,1]=[0.7843; 0.2157]
irho[:,2]=[9;10]


println("The erasure threshold is approximately $(calc_erasure_thresh(ilam,irho,40,1e5))")


The erasure threshold is approximately 0.4205844904904552


### We use the parity check matrix in example 3 given in R/U Efficient Encoding of Low-Density Parity-Check Codes

In [5]:
#since we are given a parity check matrix, we initialize a ldpcH type with H
H=zeros(Uint8,6,12)
H[1,:]=[1 1 1 0 0 1 1 0 1 0 0 0]
H[2,:]=[1 1 1 1 0 0 0 1 0 1 0 0]
H[3,:]=[0 0 0 0 1 1 1 0 1 1 1 0]
H[4,:]=[1 0 0 1 1 0 0 0 0 1 1 1]
H[5,:]=[0 1 0 1 0 0 1 1 0 0 1 1]
H[6,:]=[0 0 1 0 1 1 0 1 1 0 0 1]
n=12
LH=ldpcH(H)

#for bookkeeping we let ourselves know LH is in RU form
LH.htype="RU"
LH.ruperm=zeros(Int64,12)
#[1 2 3 4 10 6 7 5 11 12  8  9]
# 1 2 3 4  5 6 7 8  9 10 11 12
LH.ruperm[:]=[1 2 3 4 8 6 7 11 12 5 9 10]
#now we set Phi^(-1)
LH.phin=zeros(Uint8,2,2)
LH.phin[:]=[ 1 1 0 1]
LH.phin=minvmod(LH.phin,2)

#now we encode the corresponding source message as in the paper
sk=zeros(Uint8,6)
sk[:]=[1 0 0 0 0 0]
xn= ldpcRUenc(LH,sk)
println("The codeword for the original matrix is $((xn))")
txn=zeros(Uint8,n)
for i=1:LH.n
    txn[LH.ruperm[i]]=xn[i] 
end
println("Observe that (sk,p1,p2)=$((txn))")


The codeword for the original matrix is UInt8[0x01,0x00,0x00,0x00,0x01,0x00,0x00,0x01,0x00,0x00,0x01,0x00]
Observe that (sk,p1,p2)=UInt8[0x01,0x00,0x00,0x00,0x00,0x00,0x00,0x01,0x01,0x00,0x01,0x00]


### Here we initialize a ldpcH type from an alist file off of David MacKay's website

In [7]:

#download the file 96.33.964 from
# http://www.inference.phy.cam.ac.uk/mackay/codes/EN/C/96.33.964 
#then create a ldpcH type
LH=ldpcH("120.64.3.109.txt")
#now we put LH into Rudiger/Urbanke form , and choose an "optimistic" gap 
#see http://www.ldpc-codes.com/papers/encoding.pdf
optimistic_gap=5
LHru=calc_RU(LH,optimistic_gap)
println("")
#now we can encode a source vector
sk=zeros(Uint8,LH.k)
sk[1]=1

#Observe we use LHru to encode, and will use LH for message passing algorithms
xn=ldpcRUenc(LHru,sk)
println("We have encoded sk")
println("Note that xn is a codeword of LH not neccessarily LHru")
println("(Is xn a codeword of LH)=$(convert(Bool,isCW(LH,xn)))")
println("(Is xn a codeword of LHru)=$(convert(Bool,isCW(LHru,xn)))")

The theoretically achievable gap is 9.0, and the resulting gap is 8.

We have encoded sk
Note that xn is a codeword of LH not neccessarily LHru
(Is xn a codeword of LH)=true
(Is xn a codeword of LHru)=false


### Here we Generate a random (3,6) parity check matrix, and put it into RU form


In [9]:
#set channel /code parameters
n=1000
dv=3
dc=6
m=round(Int,n*dv/dc)
k=n-m
LH=ldpcH
mg=ceil(Integer,2*ell(dv,dc,n))
println("Code has rate $(k/n) ")
println("The ($dv,$dc)-code must have girth greater then $(2*ell(dv,dc,n))")

#we attempt to randomly generate a (3,6) parity check matrix.
#this might have to be run several times

#we initialize girth6=true to specify that the code will have girth > 4
girth6=true

LH = ldpcH(ones(Uint8,n)*dv,ones(Uint8,m)*dc,girth6);


if LH != false
    println("We were successfull and will continue")
    #as shown in the corresponding paper, on average using algorithm AH, the expected gap for a (3,6) code
    #is approximately  0.07*n for (3,6) LDPC code
    tgap=round(Int64,0.07*n)
    LHru=calc_RU(LH,tgap)
    if LHru != false
        println("We were successful")
    else
        println("Please rerun")

    end
    
else
    println("Please rerun")
end

Code has rate 0.5 
The (3,6)-code must have girth greater then 6.249877473216599
We were successfull and will continue
The theoretically achievable gap is 70.0, and the resulting gap is 61.
We were successful


### Here we calculate the minimum cycle length at a particular variable node, and then calculate the girth.

In [10]:
#This function returns a minimal length cycle that includes the corresponding variable node
#Note the format is odd locations are variable nodes
# and even locations are check nodes. To get the corresponding check node number
#subtract  LH.n

tem=calc_mcycle(LH,1)
println("The cycle is $tem")
println("The second element in the cycle corresponds to check node $(tem[2])-$(LH.n) = $(tem[2]-LH.n)")

#now we calculate the girth
println("The girth of LH is $(calc_girth(LH))")

The cycle is [1,1406,674,1125,157,1400]
The second element in the cycle corresponds to check node 1406-1000 = 406
The girth of LH is 6



## Using the previous initialized parity check matrix, we generate a random message, pass it through the erasure channel, and then use message passing to decode

In [11]:
sk=zeros(Uint8,LH.k)
sk[:]=rand(Bernoulli(0.5),LH.k)   #source message
#note here we encode with LHru, i.e. the parity check matrix in RU form
#we do message passing with LH, i.e., the original parity check matrix 
xn=ldpcRUenc(LHru,sk); #encode using RU method

n=LH.n
maxi=100 #maximum iterations for message passing
d=0.3 #erasure probability

yn= bec(d,xn) #pass through erasure channel, erasures are denoted by 2

cnt=0 
for i=1:n #count the # of erasures
    if yn[i]==2
        cnt+=1
    end
end
println("The # of erasures is $cnt")

xnp=bec_MPD(yn,LH,maxi) #run message passing for maxi iterations
 ne=0
rgt=0
for i=1:n 
    if xnp[i]==2
        ne+=1 
    end
    if xnp[i] == xn[i]
        rgt+=1
    end
end
println("The # of erasures after message passing decoding is $ne")
println(" n=$n #right= $rgt  BER= $(1.0-rgt/n)")


The # of erasures is 325
The # of erasures after message passing decoding is 0
 n=1000 #right= 1000  BER= 0.0


# Here we generate a random message, pass it through the binary symmetric channel, and then use message passing to decode

In [12]:
sk=zeros(Uint8,LH.k)
sk[:]=rand(Bernoulli(0.5),LH.k)   #source message
k=LH.k
maxi=1000 #maximum iterations for message passing
d=0.04 #BSC probability

xn=ldpcRUenc(LHru,sk); #encode using RU method
yn= bsc(d,xn) 

cnt=0 
for i=1:n #count the # of flips
    if yn[i] != xn[i]
        cnt+=1
    end
end
println("The # of bit flips is $cnt")

xnp=bsc_MPD(yn,d,LH,maxi) #run message passing for maxi iterations
             
ne=0
rgt=0
for i=1:n 
    if xnp[i]!=xn[i]
        ne+=1 
    else
        rgt+=1
    end
end
println("# of flips after MPD $ne")
println(" n=$n #right= $rgt  BER= $(1.0-rgt/n)")

The # of bit flips is 44
# of flips after MPD 0
 n=1000 #right= 1000  BER= 0.0


## Here we generate a random message, pass it through the BIAWGN channel, and then use message passing to decode

In [13]:
sk=zeros(Uint8,LH.k)
sk[:]=rand(Bernoulli(0.5),LH.k)   #source message
n=LH.n
maxi=500 #maximum iterations for message passing
vr=0.8^2 #AWGN Variance 

xn=zeros(Uint8,n)#ldpcRUenc(LHru,sk); #encode using RU method
yn= biawgn(vr,xn) 

cnt=0 
for i=1:n #count the # of flips
    if yn[i] <= 0  &&  xn[i]!=1
        cnt+=1
    elseif yn[i] > 0  &&  xn[i]!=0
        cnt+=1
    end
end
println("The # of bit flips is $cnt")



xnp=biawgn_MPD(yn,vr,LH,maxi) #run message passing for maxi iterations
ne=0
rgt=0
for i=1:n 
    if xnp[i]!=xn[i]
        ne+=1 
    else
        rgt+=1
    end
end
println("# of flips after MPD $ne")
println(" n=$n #right= $rgt  BER= $(1.0-rgt/n)")

The # of bit flips is 109
# of flips after MPD 0
 n=1000 #right= 1000  BER= 0.0
